<a href="https://colab.research.google.com/github/siv4gurunathan/trading_journal/blob/main/research_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# =========================
# 1️⃣ Load Data
# =========================

df = pd.read_excel("Market return_updated.xlsx")

df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')
df.set_index('Date', inplace=True)

print("Data Loaded Successfully")

# =========================
# 2️⃣ Convert RF to Decimal
# =========================

df['RF'] = df['RF'] / 100

# =========================
# 3️⃣ Define ETFs (Added XLU)
# =========================

etfs = ['XLE','XLF','XLP','XLK','XLU']

# =========================
# 4️⃣ Create Excess Returns
# =========================

for etf in etfs:
    df[etf + '_excess'] = df[etf] - df['RF']

df['SPX_excess'] = df['SPX'] - df['RF']

print("Excess returns created")

# =========================
# 5️⃣ CAPM (Using SPX)
# =========================

capm_results = []

for etf in etfs:
    X = sm.add_constant(df['SPX_excess'])
    y = df[etf + '_excess']

    model = sm.OLS(y, X).fit(cov_type='HC3')

    capm_results.append({
        'ETF': etf,
        'Market_Beta': model.params['SPX_excess'],
        'p_value': model.pvalues['SPX_excess'],
        'R_squared': model.rsquared
    })

capm_df = pd.DataFrame(capm_results)

print("\n=== CAPM Results (Using SPX) ===")
print(capm_df)

# =========================
# 6️⃣ Behavioural Beta Model
# =========================

results = []

for etf in etfs:
    X = df[['SPX_excess','SENT_ORTH']]
    X = sm.add_constant(X)
    y = df[etf + '_excess']

    model = sm.OLS(y, X).fit(cov_type='HC3')

    results.append({
        'ETF': etf,
        'Alpha': model.params['const'],
        'Market_Beta': model.params['SPX_excess'],
        'Behavioural_Beta': model.params['SENT_ORTH'],
        'Sentiment_pvalue': model.pvalues['SENT_ORTH'],
        'R_squared': model.rsquared
    })

betas_df = pd.DataFrame(results)

print("\n=== Behavioural Beta Results ===")
print(betas_df)

# =========================
# 7️⃣ Predictive Regression
# =========================

df_future = df.copy()

for etf in etfs:
    df_future[etf + '_lead'] = df_future[etf + '_excess'].shift(-1)

df_future = df_future.dropna()

predict_results = []

for etf in etfs:
    X = sm.add_constant(df_future['SENT_ORTH'])
    y = df_future[etf + '_lead']

    model = sm.OLS(y, X).fit(cov_type='HC3')

    predict_results.append({
        'ETF': etf,
        'Predict_Coeff': model.params['SENT_ORTH'],
        'p_value': model.pvalues['SENT_ORTH']
    })

predict_df = pd.DataFrame(predict_results)

print("\n=== Predictive Regression Results ===")
print(predict_df)

# =========================
# 8️⃣ High-Low Spread Test
# =========================

sorted_betas = betas_df.sort_values('Behavioural_Beta')

low_etf = sorted_betas.iloc[0]['ETF']
high_etf = sorted_betas.iloc[-1]['ETF']

print("\nLow behavioural beta ETF:", low_etf)
print("High behavioural beta ETF:", high_etf)

df['Spread'] = df[high_etf + '_excess'] - df[low_etf + '_excess']

spread_model = sm.OLS(df['Spread'], sm.add_constant(np.ones(len(df)))).fit(cov_type='HC3')

print("\n=== High-Low Spread Test ===")
print(spread_model.summary())

# =========================
# 9️⃣ Interaction (Regime Test)
# =========================

df['High_Vol'] = (df['SPX_excess'].abs() > df['SPX_excess'].std()).astype(int)

print("\n=== Interaction Model Results ===")

for etf in etfs:
    df['Interaction'] = df['SENT_ORTH'] * df['High_Vol']

    X = sm.add_constant(df[['SPX_excess','SENT_ORTH','Interaction']])
    y = df[etf + '_excess']

    model = sm.OLS(y, X).fit(cov_type='HC3')

    print("\nETF:", etf)
    print(model.summary())

# =========================
# 🔟 FF5 + SPX + Sentiment Robustness
# =========================

ff_results = []

for etf in etfs:
    X = df[['SPX_excess','SMB','HML','RMW','CMA','SENT_ORTH']]
    X = sm.add_constant(X)
    y = df[etf + '_excess']

    model = sm.OLS(y, X).fit(cov_type='HC3')

    ff_results.append({
        'ETF': etf,
        'Sentiment_Coeff': model.params['SENT_ORTH'],
        'Sentiment_pvalue': model.pvalues['SENT_ORTH'],
        'R_squared': model.rsquared
    })

ff_df = pd.DataFrame(ff_results)

print("\n=== FF5 + SPX + Sentiment Robustness ===")
print(ff_df)

# =========================
# 1️⃣1️⃣ Economic Magnitude
# =========================

print("\nStandard Deviation of Sentiment:")
print(df['SENT_ORTH'].std())
from statsmodels.stats.contrast import ContrastResults

# Stack all residuals in panel-style regression (pooled regression)

stacked_df = pd.DataFrame()

for etf in etfs:
    temp = df[['SPX_excess','SENT_ORTH']].copy()
    temp['excess'] = df[etf + '_excess']
    temp['ETF'] = etf
    stacked_df = pd.concat([stacked_df, temp])

X = sm.add_constant(stacked_df[['SPX_excess','SENT_ORTH']])
y = stacked_df['excess']

pooled_model = sm.OLS(y, X).fit(cov_type='HC3')

print(pooled_model.summary())



Data Loaded Successfully
Excess returns created

=== CAPM Results (Using SPX) ===
   ETF  Market_Beta       p_value  R_squared
0  XLE     1.059716  5.302877e-23   0.398879
1  XLF     1.180328  1.893576e-93   0.702324
2  XLP     0.547586  8.885279e-42   0.454607
3  XLK     1.284772  1.009251e-98   0.742609
4  XLU     0.501417  1.410729e-16   0.251956

=== Behavioural Beta Results ===
   ETF     Alpha  Market_Beta  Behavioural_Beta  Sentiment_pvalue  R_squared
0  XLE  0.001189     1.090227          0.012547          0.034946   0.411909
1  XLF -0.000646     1.192162          0.004867          0.157598   0.705106
2  XLP  0.002750     0.552486          0.002015          0.522289   0.456041
3  XLK  0.002507     1.269994         -0.006077          0.228401   0.746481
4  XLU  0.003236     0.503519          0.000864          0.801695   0.252130

=== Predictive Regression Results ===
   ETF  Predict_Coeff   p_value
0  XLE       0.003148  0.629537
1  XLF      -0.005702  0.275771
2  XLP      -0.00